In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, log_loss, accuracy_score
from sklearn.preprocessing import  OneHotEncoder, LabelEncoder
from sklearn.compose import make_column_transformer, make_column_selector

In [15]:
sonar = pd.read_csv("C:/Python/Cases/Sonar/Sonar.csv")
le = LabelEncoder()
y = le.fit_transform( sonar['Class'] )
X = sonar.drop('Class', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=25,
                                                    stratify=y)

In [16]:
dtc = DecisionTreeClassifier(random_state=25)
knn = KNeighborsClassifier()
nb = GaussianNB()
voting = VotingClassifier(estimators=[('TREE',dtc),('KNN',knn),('NB',nb)])
voting.fit(X_train, y_train)
y_pred = voting.predict(X_test)
print( classification_report(y_test, y_pred) )

              precision    recall  f1-score   support

           0       0.67      0.85      0.75        34
           1       0.75      0.52      0.61        29

    accuracy                           0.70        63
   macro avg       0.71      0.69      0.68        63
weighted avg       0.71      0.70      0.69        63



In [17]:
dtc1 = DecisionTreeClassifier(random_state=25)
dtc2 = DecisionTreeClassifier(random_state=25, max_depth=3)
knn1 = KNeighborsClassifier(n_neighbors=5)
knn2 = KNeighborsClassifier(n_neighbors=3)
nb = GaussianNB()
voting = VotingClassifier(estimators=[('TREE1',dtc1),('TREE2',dtc2),
                                      ('KNN1',knn1),('KNN2',knn2),
                                      ('NB',nb)])
voting.fit(X_train, y_train)
y_pred = voting.predict(X_test)
print( classification_report(y_test, y_pred) )

              precision    recall  f1-score   support

           0       0.67      0.85      0.75        34
           1       0.75      0.52      0.61        29

    accuracy                           0.70        63
   macro avg       0.71      0.69      0.68        63
weighted avg       0.71      0.70      0.69        63



Evaluating the individual estimators

In [22]:
for i in range(len(voting.estimators_)):
    print("Estimator: ", voting.estimators_[i])
    print("Accuracy Score = ", accuracy_score(y_test, voting.estimators_[i].predict(X_test)))

Estimator:  DecisionTreeClassifier(random_state=25)
Accuracy Score =  0.6984126984126984
Estimator:  DecisionTreeClassifier(max_depth=3, random_state=25)
Accuracy Score =  0.6666666666666666
Estimator:  KNeighborsClassifier()
Accuracy Score =  0.746031746031746
Estimator:  KNeighborsClassifier(n_neighbors=3)
Accuracy Score =  0.8095238095238095
Estimator:  GaussianNB()
Accuracy Score =  0.6349206349206349


`soft` voting

In [32]:
voting = VotingClassifier(estimators=[('TREE1',dtc1),('TREE2',dtc2),
                                      ('KNN1',knn1),('KNN2',knn2),
                                      ('NB',nb)], voting='soft')
voting.fit(X_train, y_train)
y_pred = voting.predict(X_test)
print( classification_report(y_test, y_pred) )
y_pred_prob = voting.predict_proba(X_test)
print( log_loss(y_test, y_pred_prob) )

              precision    recall  f1-score   support

           0       0.68      0.88      0.77        34
           1       0.79      0.52      0.62        29

    accuracy                           0.71        63
   macro avg       0.74      0.70      0.70        63
weighted avg       0.73      0.71      0.70        63

0.4923458955919867


Specifying weights

In [33]:
voting = VotingClassifier(estimators=[('TREE1',dtc1),('TREE2',dtc2),
                                      ('KNN1',knn1),('KNN2',knn2),
                                      ('NB',nb)], voting='soft',
                          weights=[7, 6, 7.5, 8.1, 6.3])
voting.fit(X_train, y_train)
y_pred = voting.predict(X_test)
print( classification_report(y_test, y_pred) )
y_pred_prob = voting.predict_proba(X_test)
print( log_loss(y_test, y_pred_prob) )

              precision    recall  f1-score   support

           0       0.68      0.88      0.77        34
           1       0.79      0.52      0.62        29

    accuracy                           0.71        63
   macro avg       0.74      0.70      0.70        63
weighted avg       0.73      0.71      0.70        63

0.4792459136222534
